# NHS Workforce Statistics Visualisation

This code downloads the latest NHS workforce statistics and visualises trends in the data

In [31]:
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

button = widgets.Button(description="Tu")
button.on_click(run_all)
display(button)

# Code to hide all code cells (useful for presentation)
hide_code = """
<script>
    var code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show code"></form>
"""
display(HTML(hide_code))


Button(description='Download latest data and visualise', style=ButtonStyle())

In [24]:
#Find URL with the latest data

import requests
from bs4 import BeautifulSoup
import re

# URL of the NHS workforce statistics page
base_url = 'https://digital.nhs.uk'
page_url = '/data-and-information/publications/statistical/nhs-workforce-statistics'

# Send a GET request
response = requests.get(base_url + page_url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all links that contain a date format typically used for these statistics
links_with_dates = soup.find_all('a', string=re.compile(r'\bJanuary|February|March|April|May|June|July|August|September|October|November|December\b \d{4}'))

# Check if we found any links and store the full URL
if links_with_dates:
    # Check if the link is absolute or relative and correct it if necessary
    latest_statistic_url = links_with_dates[0]['href']
    if not latest_statistic_url.startswith('http'):
        latest_statistic_url = base_url + latest_statistic_url
    print("Full URL to the landing page for the latest statistics:", latest_statistic_url)
else:
    print("No statistics found")

Full URL to the landing page for the latest statistics: https://digital.nhs.uk/data-and-information/publications/statistical/nhs-workforce-statistics/april-2024


In [25]:
#Find csv zip file on from the lastest statistics landing page and download


# Send a GET request to the latest statistics page
response = requests.get(latest_statistic_url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content of the latest statistics page
soup = BeautifulSoup(response.text, 'html.parser')

# Find the link to the ZIP file containing the statistics
zip_link = soup.find('a', href=lambda x: x and x.endswith('.zip'))

if zip_link:
    zip_url = zip_link['href']
    # Download the ZIP file
    if not zip_url.startswith('http'):
        zip_url = 'https://digital.nhs.uk' + zip_url  # Correct the URL if it's relative
    zip_response = requests.get(zip_url)
    zip_response.raise_for_status()  # Ensure the request was successful
    
    # Save the ZIP file locally
    with open('NHS_Workforce_Statistics.zip', 'wb') as f:
        f.write(zip_response.content)
    print("ZIP file has been downloaded successfully.")
else:
    print("No ZIP file link found on the page.")


ZIP file has been downloaded successfully.


In [26]:
#Unzip latest csv file

import zipfile
import os

# Path to the downloaded ZIP file
zip_file_path = 'NHS_Workforce_Statistics.zip'

# Directory where the contents will be extracted
extract_to_dir = 'NHS_Workforce_Statistics'

# Create the directory if it doesn't exist
if not os.path.exists(extract_to_dir):
    os.makedirs(extract_to_dir)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)

print(f"Contents extracted to {extract_to_dir}")


Contents extracted to NHS_Workforce_Statistics


In [27]:
# Reload and reprocess the data since the session was reset
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px

# Directory where the files were extracted
extracted_dir = 'NHS_Workforce_Statistics'

# Find CSV files that include "Staff Group and NHSE region" in their filename
for filename in os.listdir(extracted_dir):
    if "Staff Group and NHSE region" in filename and filename.endswith('.csv'):
        full_path = os.path.join(extracted_dir, filename)
        # Read the CSV file using pandas
        data = pd.read_csv(full_path)
        print(f"Data from {filename}:")
        break
else:
    print("No CSV file containing 'Staff Group and NHSE region' found.")

Data from NHS Workforce Statistics, April 2024 Staff Group and NHSE region.csv:


In [28]:
# Load the uploaded CSV file to reprocess the data
data = pd.read_csv(full_path)

# Pivot the data to separate FTE and HC into different columns again
df = data.pivot_table(
    index=["Date", "NHSE_Region_Code", "NHSE_Region_Name", "Staff Group Sort Order", "Staff Group"],
    columns="Data Type",
    values="Total",
    aggfunc="sum"
).reset_index()

# Rename the columns for clarity
df.columns = [col if not col in ['FTE', 'HC'] else f"{col}_Total" for col in df.columns]

# Calculate the ratio of FTE to HC
df['FTE_HC_Ratio'] = df['FTE_Total'] / df['HC_Total']

In [29]:
import dash
from dash import dcc, html, Input, Output, State  # Make sure to include Output
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)


# Layout of the app
app.layout = html.Div([
    html.H1("NHS Workforce Statistics Dashboard"),
    html.Label("Select Staff Groups:"),
    dcc.Dropdown(
        id='staff_group_selector',
        options=[{'label': i, 'value': i} for i in df['Staff Group'].unique()],
        value=['Total'],  # Default values
        multi=True
    ),
    html.Label("Select NHS Regions:"),
    dcc.Dropdown(
        id='region_selector',
        options=[{'label': i, 'value': i} for i in df['NHSE_Region_Name'].unique()],
        value=['England'],  # Default values
        multi=True
    ),
    html.Label("Select Metric:"),
    dcc.Dropdown(
        id='metric_selector',
        options=[
            {'label': 'FTE Total', 'value': 'FTE_Total'},
            {'label': 'HC Total', 'value': 'HC_Total'},
            {'label': 'FTE to HC Ratio', 'value': 'FTE_HC_Ratio'}
        ],
        value='FTE_Total'  # Default value
    ),
    dcc.Graph(id='line_chart'),
    html.H2("Data Table"),
    html.Div(id='data_table')
])

# Callback to update the line chart
@app.callback(
    Output('line_chart', 'figure'),
    [Input('staff_group_selector', 'value'),
     Input('region_selector', 'value'),
     Input('metric_selector', 'value')]
)
def update_graph(selected_staff_groups, selected_regions, selected_metric):
    filtered_df = df[df['Staff Group'].isin(selected_staff_groups) & df['NHSE_Region_Name'].isin(selected_regions)]
    fig = px.line(filtered_df, x='Date', y=selected_metric, color='Staff Group', title=f"{selected_metric} over Time")
    fig.update_layout(
        plot_bgcolor='white',
        xaxis_title='Date',
        yaxis_title=selected_metric,
        legend_title_text='Staff Group',
        xaxis_showgrid=False,  # Hide the gridlines (optional)
        yaxis_showgrid=False,  # Hide the gridlines (optional)
        xaxis_linecolor='black',  # X-axis line color
        yaxis_linecolor='black',  # Y-axis line color
        xaxis_linewidth=2,  # X-axis line width
        yaxis_linewidth=2   # Y-axis line width
    )
    return fig

# Callback to update the data table
@app.callback(
    Output('data_table', 'children'),
    [Input('staff_group_selector', 'value'),
     Input('region_selector', 'value')]
)
def update_table(selected_staff_groups, selected_regions):
    filtered_df = df[df['Staff Group'].isin(selected_staff_groups) & df['NHSE_Region_Name'].isin(selected_regions)]
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in filtered_df.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(filtered_df.iloc[i][col]) for col in filtered_df.columns
            ]) for i in range(min(len(filtered_df), 10))  # Display only the first 10 rows
        ])
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)